In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

from sklearn.model_selection import train_test_split

import tqdm

In [18]:
DATA_PATH = '../data/raw/labeled_texts_fixed_binary.csv'
DATA_DIR = '../data/raw'

In [3]:
MODEL_PATH = '../models/word2vec_embeddings/cc.ru.300.vec'

In [6]:
df = pd.read_csv(DATA_PATH)

In [8]:
df.head()

,texts,labels,nationality,families,sex,limit,dd,joined
0,Сдаётся студия на длительный срок в доме комфо...,Нет особенностей,0,0,0,0,"['0', '0', '0', '0']",0
1,Раздвижной шифоньер. Сдам на долгий срок 1-к. ...,Нет особенностей,0,0,0,0,"['0', '0', '0', '0']",0
2,Предлагается в аренду студия 30 кв.м. в СНТ Да...,Нет особенностей,0,0,0,0,"['0', '0', '0', '0']",0
3,"Сдается замечательная, светлая, просторная одн...",Нет особенностей,0,0,0,0,"['0', '0', '0', '0']",0
4,"МЕТРО 5 мин пешком!Сдается уютная , светлая кв...",Предпочтение славянам,1,0,0,0,"['1', '0', '0', '0']",1000


# Национальность

In [9]:
nationality_df = df[['texts', 'nationality']]

In [11]:
nationality_df.head()

,texts,nationality
0,Сдаётся студия на длительный срок в доме комфо...,0
1,Раздвижной шифоньер. Сдам на долгий срок 1-к. ...,0
2,Предлагается в аренду студия 30 кв.м. в СНТ Да...,0
3,"Сдается замечательная, светлая, просторная одн...",0
4,"МЕТРО 5 мин пешком!Сдается уютная , светлая кв...",1


In [16]:
train_nationality_df, test_nationality_df = train_test_split(nationality_df, test_size=0.3,
                                                             stratify=nationality_df['nationality'])

In [23]:
train_nationality_df = train_nationality_df.reset_index(drop=True)
test_nationality_df = test_nationality_df.reset_index(drop=True)

In [ ]:
test_nationality_df.to_csv(DATA_DIR + 'test_nationality.csv', index=False)

In [24]:
train_nationality_df.head()

,texts,nationality
0,Прекрасный угловой апартамент в потрясающем не...,0
1,"Сдается студия, Стиральная машина в коридоре....",1
2,"Moscow City! Высокий этаж, не забываемый вид н...",0
3,Кухня-гостиная и две изолированные спальни. По...,0
4,ID 44193 В аренду предлагается стильная 4-комн...,0


In [19]:
train_nationality_df['nationality'].sum() / len(train_nationality_df)

0.10286399760083971

In [20]:
test_nationality_df['nationality'].sum() / len(test_nationality_df)

0.10283315844700944

In [35]:
pos_labels_fraction = test_nationality_df['nationality'].sum() / len(test_nationality_df)
labels_ratio = (1 - pos_labels_fraction) / pos_labels_fraction
n_texts_to_generate = int(labels_ratio)

In [27]:
positive_texts = train_nationality_df[train_nationality_df['nationality'] == 1]['texts'].values

In [42]:
%%time
aug = naw.WordEmbsAug(model_type='fasttext', model_path=MODEL_PATH, action="substitute", top_k=10,
                      stopwords=['не', 'или', 'и'])

CPU times: total: 0 ns
Wall time: 0 ns


In [70]:
augmented_texts = aug.augment(positive_texts[0].lower(), n=10)
print("Original:")
print(positive_texts[0])
print("Augmented Texts:")
print(augmented_texts)

Original:
Сдается студия,  Стиральная машина в коридоре. В студии шкаф, стол,кровать, кухня, сан.узел. На длителтный срок. Без привода гостей. Одну женщину, спокойную, адекватную, РФ. Собственница проживает в соседней студии. Риелторам просьба не беспокоить. Пишите сообщения.Залог 15тыс. Пишите, не всегда могу ответить на звонки. Пожалуйста читайте об'явление - только женщине, одной! Это значит НЕ СЕМЕЙНОЙ ПАРЕ , НЕ МУЖЧИНАМ !!!
Augmented Texts:
["сдается студия, стиральная машина в коридоре. 5.В студии шкаф-купе, стульчик, кровать, кухня, сан.. узел. на длителтный срок. без привода гостей. одну жену, тихую, разумную, рф. собственница проживает в соседней студии. риелторам просьба не беспокоить. пишите сообщения. залогу 15тыс. пишите, не всегда могу ответить на телефонные. пожалуйста читайте об ' явление - только женщине, одной! это значит не семейной паре, не дамам! !!", "сдается студия, стиральная машина- в коридоре. в студии шкаф, столу, кровать, кухня, сане. узел. Hа длителтный сро

In [44]:
%%time
augmented_texts = []
for text in positive_texts:
    augmented_texts += aug.augment(text, n=n_texts_to_generate)

KeyboardInterrupt: 

In [61]:
len(augmented_texts)

1

In [58]:
augmented_texts_df = pd.DataFrame({'texts': augmented_texts, 'nationality': [1] * len(augmented_texts)})

In [59]:
augmented_texts_df['texts'][20:25].values

array([], dtype=object)

In [62]:
concatenated_nationality_df = pd.concat([train_nationality_df, augmented_texts_df])

In [64]:
concatenated_nationality_df

,texts,nationality
0,Прекрасный угловой апартамент в потрясающем не...,0
1,"Сдается студия, Стиральная машина в коридоре....",1
2,"Moscow City! Высокий этаж, не забываемый вид н...",0
3,Кухня-гостиная и две изолированные спальни. По...,0
4,ID 44193 В аренду предлагается стильная 4-комн...,0
...,...,...
6665,Лот 551403. Бонус коллегам 35.000р! Предлагает...,0
6666,Сдаётся 2х комнатная квартира в отличном состо...,1
6667,"Сдаётся однокомнатная квартира, ул. Новолесная...",0
6668,Сдается 2-х комнатная квартира в шаговой досту...,1


In [67]:
concatenated_nationality_df = pd.concat([train_nationality_df,
                                         augmented_texts_df]) \
                                        .reset_index(drop=True)